In [52]:
from helper_dask import launch_dask_cluster
import runhouse as rh 

num_nodes = 3
cluster_name = f"py-{num_nodes}"

cluster = rh.cluster(
    name=cluster_name,
    instance_type="r5d.xlarge",
    num_instances=num_nodes,
    provider="aws",
).up_if_not()

launch_dask_cluster(cluster)

try:
    while True:
        pass
except KeyboardInterrupt:
    print("Tore down Dask cluster")

INFO | 2024-11-05 14:37:02 | runhouse.resources.hardware.cluster:1511 | Running command on py-3: echo "hello"


Output()

INFO | 2024-11-05 14:37:10 | runhouse.resources.hardware.cluster:1511 | Running command on py-3: echo "hello"
INFO | 2024-11-05 14:37:12 | runhouse.resources.hardware.cluster:1511 | Running command on py-3: echo "hello"
INFO | 2024-11-05 14:37:13 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32300 is already in use. Trying next port.
INFO | 2024-11-05 14:37:13 | runhouse.resources.hardware.ssh_tunnel:250 | Port 32301 is already in use. Trying next port.
INFO | 2024-11-05 14:37:13 | runhouse.resources.hardware.ssh_tunnel:91 | Running forwarding command: ssh -T -L 32302:localhost:32300 -i ~/.ssh/sky-key -o Port=22 -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null -o LogLevel=ERROR -o IdentitiesOnly=yes -o ExitOnForwardFailure=yes -o ServerAliveInterval=5 -o ServerAliveCountMax=3 -o ConnectTimeout=30s -o ForwardAgent=yes -o ControlMaster=auto -o ControlPath=/tmp/skypilot_ssh_6ba66db0/f516e5f64e/%C -o ControlPersist=300s ubuntu@3.89.128.11
WARNING | 2024-11-05 14:37:13 | ru